In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import enchant
from matplotlib import pyplot as plt


# 参数

In [2]:
# 生成一个查词器
dic = enchant.Dict("en_US")
batch_size = 64
lr_g = 0.0001
lr_d = 0.0001
noise = 1
word_len = 5
x = torch.unsqueeze(torch.linspace(-1,1,100),dim=1) #因为在torch中它的数据是有维度的，unsqueeze就是把一维的数据转化为二维
y = x.pow(2) + 0.2*torch.rand(x.size())
x,y = Variable(x), Variable(y)

# 读取数据

In [3]:
# 压缩到（-1，1）范围内
def compose(x):
    def cp(z):
        return (x-97) * (2.0/26) - 1    
    for wd in x:
        wd[0] = cp(wd[0])
        wd[1] = cp(wd[1])
        wd[2] = cp(wd[2])
        wd[3] = cp(wd[3])
        wd[4] = cp(wd[4])
    
    return x
# 解压缩
def decompose(x):
    rst = 97 + (x + 1) / (2.0/26)
    if rst > 122:
        rst = 122
    return rst
    
def proper_word():
    # 使用 97-112(ascii) 表示字母 a-z
    # 读取 5字母词语字典
    wordList = [line.rstrip('\n') for line in open("sgb-words.txt")]
    wordnp = np.vstack([ord(w[0]),ord(w[1]),ord(w[2]),ord(w[3]),ord(w[4])] for w in wordList)
    p_word = torch.from_numpy(wordnp[0:5696]).float()
    p_word = iter(np.split(p_word,89))
    return Variable(next(p_word))

# 神经网络

In [4]:
# Generator
G = nn.Sequential(                      
    nn.Linear(noise, 128),
    nn.ReLU(),
    nn.Linear(128, word_len),
    nn.Tanh()
)

#Discriminator
D = nn.Sequential(                      
    nn.Linear(word_len, 128),     
    nn.ReLU(),
    nn.Linear(128, 1),
    nn.Sigmoid(),
)

class Net(torch.nn.Module):
	def __init__(self, n_feature,n_hidden,n_output):
		super(Net, self).__init__()
		self.hidden = torch.nn.Linear(n_feature,n_hidden)
		self.predict = torch.nn.Linear(n_hidden,n_output)
	def forward(self,x):
		x = self.hidden(x)
		x = self.predict(x)
		return x

net = Net(1,10,1)
print(net)

Net(
  (hidden): Linear(in_features=1, out_features=10, bias=True)
  (predict): Linear(in_features=10, out_features=1, bias=True)
)


In [5]:
#优化器
opt_D = torch.optim.Adam(D.parameters(), lr=lr_d)
opt_G = torch.optim.Adam(G.parameters(), lr=lr_g)

optimizer = torch.optim.SGD(net.parameters(), lr=0.5)

In [6]:

def plot_grad_flow(named_parameters):
    '''Plots the gradients flowing through different layers in the net during training.
    Can be used for checking for possible gradient vanishing / exploding problems.
    
    Usage: Plug this function in Trainer class after loss.backwards() as 
    "plot_grad_flow(self.model.named_parameters())" to visualize the gradient flow'''
    ave_grads = []
    max_grads= []
    layers = []
    for n, p in named_parameters:
        if(p.requires_grad) and ("bias" not in n):
            layers.append(n)
            ave_grads.append(p.grad.abs().mean())
            max_grads.append(p.grad.abs().max())
    plt.bar(np.arange(len(max_grads)), max_grads, alpha=0.1, lw=1, color="c")
    plt.bar(np.arange(len(max_grads)), ave_grads, alpha=0.1, lw=1, color="b")
    plt.hlines(0, 0, len(ave_grads)+1, lw=2, color="k" )
    plt.xticks(range(0,len(ave_grads), 1), layers, rotation="vertical")
    plt.xlim(left=0, right=len(ave_grads))
    plt.ylim(bottom = -0.001, top=0.02) # zoom in on the lower gradient regions
    plt.xlabel("Layers")
    plt.ylabel("average gradient")
    plt.title("Gradient flow")
    plt.grid(True)
    plt.legend([Line2D([0], [0], color="c", lw=4),
                Line2D([0], [0], color="b", lw=4),
                Line2D([0], [0], color="k", lw=4)], ['max-gradient', 'mean-gradient', 'zero-gradient'])

# 训练开始

In [7]:
for step in range(200):
    #D训练
    real_data = proper_word()
    g_noise = Variable(torch.randn(batch_size, noise))
    fake_data = G(g_noise)
    
    prob_real = D(real_data)            # D try to max this probability
    prob_fake = D(fake_data)            # D try to min this probability
    
    D_loss = - torch.mean(torch.log(prob_real) + torch.log(1. - prob_fake))
    # D_loss = - torch.mean(torch.log(prob_real))
    
    
    opt_D.zero_grad()
    D_loss.backward(retain_graph=True)  # D 反向传播 保留参数到下一次反向传播
    
    # plot_grad_flow(D.named_parameters())
   
    
    opt_D.step()
    
    # G 训练
    fake_data = G(g_noise)
    
    prob_fake = D(fake_data)            # G try to max this probability
    G_loss = torch.mean(torch.log(1. - prob_fake))
    
    opt_G.zero_grad()
    G_loss.backward()                   # G反向传播
    opt_G.step()
    print(G_loss.grad)
    
    #可视化
    # print("-", end = "")
    if step%1000 == 0:
        #转化为字母
        final_word = ""
        for letter in fake_data.data.numpy()[42]:
            asci_i = decompose(letter)
            
            final_word = final_word + chr(int(round(asci_i)))
        print()
        print()
        print("generated word:")
        print(final_word)
        print()
        print("dictionary check:")
        print(dic.check(final_word))
        print()

C:\Users\81523\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


None


generated word:
ohnnp

dictionary check:
False

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


KeyboardInterrupt: 